In [12]:
import sys
import numpy as np
import sklearn.linear_model as skl
import argparse
import os
import pickle
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
import pandas as pd


In [13]:
import sys
import argparse

# Check if the script is running in a Jupyter Notebook
if 'ipykernel' in sys.modules:
    # Default values for Jupyter Notebook
    sub = 1  # Set the default value for "sub" here
else:
    # Argument parser setup for command-line usage
    parser = argparse.ArgumentParser(description='Argument Parser')
    parser.add_argument(
        "-sub", "--sub",  # Command-line argument name
        help="Subject Number",  # Description of the argument
        default=1  # Default value if no argument is provided
    )
    args = parser.parse_args()  # Parse the command-line arguments
    sub = int(args.sub)  # Convert the argument to an integer
    # Ensure the provided value for "sub" is in the allowed set
    assert sub in [1, 2, 5, 7]


In [15]:
import numpy as np
from sklearn.linear_model import Ridge

# Subject number
sub = 1

# Load and preprocess train and test fMRI data
train_path = r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Data\processed_nsddata\subj01\Default\nsd_train_fmriavg_Default_sub{:d}.npy'.format(sub, sub)
train_fmri = np.load(train_path)

test_path = r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Data\processed_nsddata\subj01\Default\nsd_test_fmriavg_Default_sub{:d}.npy'.format(sub, sub)
test_fmri = np.load(test_path)

# Scale the fMRI data
train_fmri = train_fmri / 300
test_fmri = test_fmri / 300

# Normalize the fMRI data
norm_mean_train = np.mean(train_fmri, axis=0)
norm_scale_train = np.std(train_fmri, axis=0, ddof=1)
train_fmri = (train_fmri - norm_mean_train) / norm_scale_train
test_fmri = (test_fmri - norm_mean_train) / norm_scale_train

# Debugging information
print(np.mean(train_fmri), np.std(train_fmri))
print(np.mean(test_fmri), np.std(test_fmri))
print(np.max(train_fmri), np.min(train_fmri))
print(np.max(test_fmri), np.min(test_fmri))

# Basic info about data
num_voxels, num_train, num_test = train_fmri.shape[1], len(train_fmri), len(test_fmri)


8.420516909861324e-17 0.9999435586284282
-0.0005780503836415376 0.9879594513515076
18.847649652192835 -26.616083787460692
12.224709200013582 -11.162228385394327


In [16]:
train_fmri.shape

(8859, 23616)

In [17]:
# Define the base path to the extracted features
base_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results"
save_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results"

# Ensure the save directory exists
os.makedirs(save_path, exist_ok=True)

def process_layer(layer, degree):
    print(f"Processing layer {layer} with degree {degree}...")

    # Ensure that the fmri data is in float32 (assuming they are defined globally)
    global train_fmri, test_fmri
    train_fmri = train_fmri.astype(np.float32)
    test_fmri = test_fmri.astype(np.float32)

    # Path to the layer's latent features
    layer_path = f"{base_path}\\extracted_fetures_{layer}_Layered\\subj{sub:02d}\\nsd_vdvae_features_{layer}l.npz"
    nsd_features = np.load(layer_path)
    train_latents = nsd_features['train_latents'].astype(np.float32)
    test_latents = nsd_features['test_latents'].astype(np.float32)

    def expand_features(X, degree):
        # Ensure X is float32 before expansion
        X = X.astype(np.float32)
        if degree < 2:
            return X
        expanded = [X]
        for d in range(2, degree + 1):
            # Each power is cast back to float32
            expanded.append((X ** d).astype(np.float32))
        return np.hstack(expanded).astype(np.float32)

    X_train_poly = expand_features(train_fmri, degree)
    X_test_poly = expand_features(test_fmri, degree)

    # Initialize these variables so they exist even if an error occurs
    pred_test_latent = None
    std_norm_test_latent = None
    pred_latents = None

    try:
        print("Training latents Feature Regression")
        reg = Ridge(alpha=50000, max_iter=10000, fit_intercept=True)
        reg.fit(X_train_poly, train_latents)

        # Predict and cast to float32
        pred_test_latent = reg.predict(X_test_poly).astype(np.float32)

        # Compute mean and std in float32
        pred_mean = np.mean(pred_test_latent, axis=0).astype(np.float32)
        pred_std = np.std(pred_test_latent, axis=0).astype(np.float32)
        std_norm_test_latent = ((pred_test_latent - pred_mean) / pred_std).astype(np.float32)

        train_mean = np.mean(train_latents, axis=0).astype(np.float32)
        train_std = np.std(train_latents, axis=0).astype(np.float32)
        pred_latents = (std_norm_test_latent * train_std + train_mean).astype(np.float32)

        r2_score = reg.score(X_test_poly, test_latents)
        print(f"R^2 Score on test set: {r2_score}")

        degree_folder = os.path.join(save_path, f"degree{degree}")
        os.makedirs(degree_folder, exist_ok=True)

        layer_save_path = os.path.join(degree_folder, f"layer_{layer}_regression.npz")
        np.savez(layer_save_path,
                 coefficients=reg.coef_.astype(np.float32),
                 intercept=reg.intercept_.astype(np.float32),
                 predicted_test_latents=pred_latents)

        weights_path = os.path.join(degree_folder, f"layer_{layer}_regression_weights.pkl")
        datadict = {"weight": reg.coef_.astype(np.float32),
                    "bias": reg.intercept_.astype(np.float32)}
        with open(weights_path, "wb") as f:
            pickle.dump(datadict, f)

        r2_score_file = os.path.join(degree_folder, "r2_scores.txt")
        with open(r2_score_file, "a") as f:
            f.write(f"Layer {layer}: R^2 Score: {r2_score}\n")

        print(f"Saved results for layer {layer} in folder {degree_folder}")

    finally:
        # Free up memory by deleting variables if they exist
        del train_latents
        del test_latents
        if pred_test_latent is not None:
            del pred_test_latent
        if std_norm_test_latent is not None:
            del std_norm_test_latent
        if pred_latents is not None:
            del pred_latents
        print(f"Freed up memory for layer {layer}")


In [18]:
# [1, 2, 3, 4, 5, 7, 9, 11, 13, 15, 25, 35, 45, 55, 65]
latent_sizes = [31]
#[1, 2, 3, 4, 5]
degrees = [1, 2, 3, 4]  # List of degrees to test

for degree in degrees:
    for layer in latent_sizes:
        process_layer(layer, degree)


Processing layer 31 with degree 1...
Training latents Feature Regression
R^2 Score on test set: -0.03864651173353195
Saved results for layer 31 in folder C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree1
Freed up memory for layer 31
Processing layer 31 with degree 2...
Training latents Feature Regression
R^2 Score on test set: -0.10864412039518356
Saved results for layer 31 in folder C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree2
Freed up memory for layer 31
Processing layer 31 with degree 3...
Training latents Feature Regression
R^2 Score on test set: -0.5892534255981445
Saved results for layer 31 in folder C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree3
Freed up memory for layer 31
Processing layer 31 with degree 4...
Training latents Feature Regression


C:\Users\sOrOush\.conda\envs\linearity_question\Lib\site-packages\sklearn\linear_model\_ridge.py:253: LinAlgWarning: Ill-conditioned matrix (rcond=3.28241e-08): result may not be accurate.
  dual_coef = linalg.solve(K, y, assume_a="pos", overwrite_a=False)


R^2 Score on test set: -3.0745909214019775
Saved results for layer 31 in folder C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree4
Freed up memory for layer 31


In [7]:
import numpy as np

# Replace with the correct file path for the output file
npz_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree1\layer_31_regression.npz"

# Load the npz file
data = np.load(npz_path)

# Print out the shapes of the saved arrays
print("Coefficients shape:", data['coefficients'].shape)         # Expected: (L, F * degree)
print("Intercept shape:", data['intercept'].shape)               # Expected: (L,)
print("Predicted Test Latents shape:", data['predicted_test_latents'].shape)  # Expected: (N_test, L)


Coefficients shape: (91168, 2558)
Intercept shape: (91168,)
Predicted Test Latents shape: (982, 91168)


In [8]:
import numpy as np

# Replace with the correct file path for the output file
npz_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree2\layer_31_regression.npz"

# Load the npz file
data = np.load(npz_path)

# Print out the shapes of the saved arrays
print("Coefficients shape:", data['coefficients'].shape)         # Expected: (L, F * degree)
print("Intercept shape:", data['intercept'].shape)               # Expected: (L,)
print("Predicted Test Latents shape:", data['predicted_test_latents'].shape)  # Expected: (N_test, L)


Coefficients shape: (91168, 5116)
Intercept shape: (91168,)
Predicted Test Latents shape: (982, 91168)


In [9]:
import numpy as np

# Replace with the correct file path for the output file
npz_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree3\layer_31_regression.npz"

# Load the npz file
data = np.load(npz_path)

# Print out the shapes of the saved arrays
print("Coefficients shape:", data['coefficients'].shape)         # Expected: (L, F * degree)
print("Intercept shape:", data['intercept'].shape)               # Expected: (L,)
print("Predicted Test Latents shape:", data['predicted_test_latents'].shape)  # Expected: (N_test, L)


Coefficients shape: (91168, 7674)
Intercept shape: (91168,)
Predicted Test Latents shape: (982, 91168)


In [10]:
import numpy as np

# Replace with the correct file path for the output file
npz_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree4\layer_31_regression.npz"

# Load the npz file
data = np.load(npz_path)

# Print out the shapes of the saved arrayws
print("Coefficients shape:", data['coefficients'].shape)         # Expected: (L, F * degree)
print("Intercept shape:", data['intercept'].shape)               # Expected: (L,)
print("Predicted Test Latents shape:", data['predicted_test_latents'].shape)  # Expected: (N_test, L)


Coefficients shape: (91168, 10232)
Intercept shape: (91168,)
Predicted Test Latents shape: (982, 91168)


In [11]:
import numpy as np

# Replace with the correct file path for the output file
npz_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\polynomial_regression_results\degree5\layer_31_regression.npz"

# Load the npz file
data = np.load(npz_path)

# Print out the shapes of the saved arrayws
print("Coefficients shape:", data['coefficients'].shape)         # Expected: (L, F * degree)
print("Intercept shape:", data['intercept'].shape)               # Expected: (L,)
print("Predicted Test Latents shape:", data['predicted_test_latents'].shape)  # Expected: (N_test, L)


Coefficients shape: (91168, 12790)
Intercept shape: (91168,)
Predicted Test Latents shape: (982, 91168)
